In [2]:
import torch
import torch.nn as nn
from res.plot_lib import set_default, show_scatterplot, plot_bases
from matplotlib.pyplot import plot, title, axis
import matplotlib.pyplot as plt
import numpy as np

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Inverse Matrices, Column Space, Rank, and Null Space

Matrix Algebra is most useful when solving *linear* systems of equations, such as:

$$
\begin{align}
2x + 4y + -2z &= 3 \\
4x + 9y + -3z &= 8 \\
-2x - 3y + 7z &= 10
\end{align}
$$

Which is simply a list of equations with unkown variables. These linear system of equations can be packaged into a single vector equation where:
 - Matrix contains coefficients $A$
 - Vector contains variables $\hat{x}$
 - Output vector (matrix-vector output) $\hat{b}$

$$
\underbrace{\begin{pmatrix} 2 & 4 & -2 \\ 4 & 9 & -3 \\ -2 & -3 & 7 \end{pmatrix}}_{\mathbf{A}}
\underbrace{\begin{pmatrix} x \\ y \\ z \end{pmatrix}}_{\mathbf{x}} =
\underbrace{\begin{pmatrix} 3 \\ 8 \\ 10 \end{pmatrix}}_{\mathbf{b}}
$$

Solving the system is saying "we're looking for a vector $\vec{x}$, which after applying the transformation $A$, lands on the vector $\vec{b}$" 

With the inverse matrix, $A^-1$, you will find the vector $\vec{x}$, that when applied to $A$, gets you to $\vec{b}$ 

Note that $\mathbf{A}^{-1}\mathbf{A} = \mathbf{I}$, where $\mathbf{I}$ is the **identity matrix**:

_Important_: When the det(A) = 0, you cannot find the inverse (you cannot find the inverse a line or plane)

$$
\mathbf{I}_{n \times n} =
\begin{pmatrix} 
1 & 0 & ... & 0 \\
0 & 1 & ... & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & ... &1 \\
\end{pmatrix}
$$

Such that

$$
\begin{align}
\mathbf{A} \mathbf{x} &= \mathbf{b} \\
\mathbf{A^{-1}}\mathbf{A}\mathbf{x} &= \mathbf{A^{-1}}\mathbf{b} \\
\mathbf{x} &= \mathbf{A^{-1}}\mathbf{b}
\end{align}
$$

### Identity Matrix

The identity matrix $I_n$ is a special matrix of shape ($n \times n$) that is filled with $0$ except the diagonal that is filled with 1.
 - An identity matrix times some vector gives you that same vector as output

In [4]:
# Identity matrix multiplied with a vector results in the same vector
x = np.array([[2], [3], [6]])
x

array([[2],
       [3],
       [6]])

In [5]:
# Identity matrix
I = np.eye(x.shape[0])
I

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [6]:
# Identity * x = x
I.dot(x)

array([[2.],
       [3.],
       [6.]])

### Inverse Matrix

It is the matrix that results in the identity matrix when it is multiplied by $A$

This means that if we apply a linear transformation to the space with $A$, it is possible to go back with $A^{-1}$. It provides a way to **cancel the transformation**
 - $A^{-1}A$ = $I$

_Critical_ - The inverse of matrices can be very useful, for instance, to solve a set of linear equations

In [7]:
A = np.array([[3, 0, 2], [2, 0, -2], [0, 1, 1]])
A

array([[ 3,  0,  2],
       [ 2,  0, -2],
       [ 0,  1,  1]])

In [8]:
# Compute the inverse
A_inv = np.linalg.inv(A)
A_inv

array([[ 0.2,  0.2,  0. ],
       [-0.2,  0.3,  1. ],
       [ 0.2, -0.3, -0. ]])

In [9]:
A_bis = A_inv.dot(A)
A_bis

array([[ 1.00000000e+00,  0.00000000e+00, -1.11022302e-16],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

### Column Space and Rank

#### Rank

When the output of a transformation is (det.) is:
 - a line, then the transformation's rank = 1
 - a plane, then the transformation's rank = 2

You can think of the rank as the # of dimensions in the output resulting from a transformation:
 - A 3d transformation's det. != 0, then rank = 3
 - A 2d transformation's det. != 0, then rank = 2
 
**Full Rank** - All rows and columns are linearly independent


#### Column Space

The set of all possible outputs for a matrix is called the column space of that matrix
 - Column space = Where $\hat{i}, \hat{j}$ lands

Null space "kernel" is a set of space where all vectors "land"
 - If vectors land on 0, then this is the **null space**
     - Null space contains all possible solutions to the equation


### Solving a System of Linear Equations

$$
\underbrace{\begin{pmatrix} 2 & 4 & -2 \\ 4 & 9 & -3 \\ -2 & -3 & 7 \end{pmatrix}}_{\mathbf{A}}
\underbrace{\begin{pmatrix} x \\ y \\ z \end{pmatrix}}_{\mathbf{x}} =
\underbrace{\begin{pmatrix} 3 \\ 8 \\ 10 \end{pmatrix}}_{\mathbf{b}}
$$

In [27]:
# By "hand" first - kind of
A = np.array([[2, 4, -2],
              [4, 9, -3],
              [-2, -3, 7]])

b = np.array([3, 8, 10])

# Find the inverse of A
invA = np.linalg.inv(A)

# Take invA and multiply it by b column vector
AB = invA.dot(b)
AB.reshape(-1, 1)

array([[ 5.75],
       [-0.75],
       [ 2.75]])

In [26]:
# Using packages
x = np.linalg.solve(A, b)
x.reshape(-1, 1)

array([[ 5.75],
       [-0.75],
       [ 2.75]])

In [ ]:
# With PyTorch
tensorA = torch.tensor([[2, 4, -2],
                        [4, 9, -3],
                        [-2, -3, 7]]) 

tensorb = torch.tensor([3, 8, 10])

invA = torch.linalg.inv(tensorA)
x = torch.linalg.solve(tensorA, tensorb)